In [1]:
import pandas as pd
import numpy as np

from alpha101 import Alpha101 

In [3]:
prices = pd.read_csv("C:/Users/z1883/Desktop/507_Final_Project/Project Code/NYC Stock Price/all_stocks_5yr.csv")

prices['date'] = pd.to_datetime(prices['date'])
prices = prices.sort_values(['Name', 'date'])

# We do some data cleaning, replace NA values to 0
prices['returns'] = prices.groupby('Name')['close'].pct_change()
prices['returns'] = prices['returns'].replace([np.inf, -np.inf], np.nan)
prices['returns'] = prices['returns'].fillna(0.0)

# calculate the Volume-weighted average price
tp = (prices['high'] + prices['low'] + prices['close']) / 3.0
prices['vwap'] = (tp * prices['volume']) / prices['volume']

prices.head()

,date,open,high,low,close,volume,Name,returns,vwap
71611,2013-02-08,45.07,45.35,45.00,45.08,1824755,A,0.000000,45.143333
71612,2013-02-11,45.17,45.18,44.45,44.60,2915405,A,-0.010648,44.743333
71613,2013-02-12,44.81,44.95,44.50,44.62,2373731,A,0.000448,44.690000
71614,2013-02-13,44.81,45.24,44.68,44.75,2052338,A,0.002913,44.890000
71615,2013-02-14,44.72,44.78,44.36,44.58,3826245,A,-0.003799,44.573333


In [5]:
# This function is used to convert the long-format price DataFrame into a
# date × stock name for a single field (e.g., 'close', 'open', etc.).
def make_panel_field(df, field):
    return df.pivot(index='date', columns='Name', values=field).sort_index()

close  = make_panel_field(prices, 'close')
open_  = make_panel_field(prices, 'open') 
high   = make_panel_field(prices, 'high')
low    = make_panel_field(prices, 'low')
volume = make_panel_field(prices, 'volume')
returns = make_panel_field(prices, 'returns')
vwap   = make_panel_field(prices, 'vwap')

volume = prices.pivot(index='date', columns='Name', values='volume').sort_index()

# Since some later formulas involve log(volume), replace zeros with 1
# to avoid taking log(0), which would be negative infinity.
volume = volume.replace(0, 1)

In [7]:
close.head(3)

Name,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2013-02-08,45.08,14.75,78.90,67.8542,36.25,46.89,34.41,73.31,39.12,45.70,...,28.24,37.51,88.61,42.87,31.84,27.09,65.30,75.85,24.14,33.05
2013-02-11,44.60,14.46,78.39,68.5614,35.85,46.76,34.26,73.07,38.64,46.08,...,28.31,37.46,88.28,42.84,31.96,27.46,64.55,75.65,24.21,33.26
2013-02-12,44.62,14.27,78.60,66.8428,35.42,46.96,34.30,73.37,38.89,46.27,...,28.41,37.58,88.46,42.87,31.84,27.95,64.75,75.44,24.49,33.74


In [9]:
high.head(3)

Name,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2013-02-08,45.35,15.12,79.72,68.4014,36.42,46.895,34.66,73.710,39.45,45.90,...,28.74,37.630,88.80,42.88,32.12,27.64,65.49,75.99,24.21,33.48
2013-02-11,45.18,15.01,78.91,69.2771,36.18,47.000,34.49,73.270,39.05,46.14,...,28.54,37.575,88.51,42.90,32.02,27.53,65.19,75.98,24.30,33.50
2013-02-12,44.95,14.51,78.63,68.9114,35.90,47.050,34.50,73.495,39.13,46.35,...,28.61,37.710,88.62,42.99,32.12,28.10,65.06,75.82,24.57,34.00


In [11]:
df_data = {
    'close': close,
    'open': open_,
    'high': high,
    'low': low,
    'volume': volume,
    'returns': returns,
    'vwap': vwap,
}

# Using the alpha function we write to calculate the alpha factor values.
alpha = Alpha101(df_data)

In [13]:
# This step is selecting alpha factors with sufficiently many valid values.
# For each candidate alpha factor function in 'alpha_names', we:
# First, compute the factor values using the price data as input.
# Second, compute the proportion of outputs that are finite (not NaN or Infinite).
# Third, if this proportion is below a predefined threshold, we discard the factor, otherwise, we keep it as a usable factor.

# This step ensures that we only keep alpha factors that behave
# reasonably well numerically before using them in the subsequent modeling.

alpha_names = [
    'alpha001', 'alpha002', 'alpha003', 'alpha004',
    'alpha006', 'alpha007', 'alpha008', 'alpha011', 
    'alpha012', 'alpha013', 'alpha014', 'alpha015', 
    'alpha016', 'alpha017', 'alpha018', 'alpha019',
    'alpha020', 'alpha022', 'alpha025', 'alpha026',
    'alpha028', 'alpha029', 'alpha030', 'alpha032', 
    'alpha033', 'alpha034', 'alpha035', 'alpha037', 
    'alpha038', 'alpha040', 'alpha041', 'alpha042', 
    'alpha043', 'alpha044', 'alpha052', 'alpha053', 
    'alpha054', 'alpha060', 'alpha101',
]

usable = {}   
bad    = {}  

for name in alpha_names:
    func = getattr(alpha, name)
    try:
        fac = func()
        arr = fac.to_numpy()
        finite_ratio = np.isfinite(arr).mean()
        
        # Threhold = 0.8
        if finite_ratio < 0.8:  
            bad[name] = f"The effective value ratio is too low：{finite_ratio:.2%}"
            print(f"{name}: delete ({bad[name]})")
            continue

        usable[name] = fac
        print(f"{name}: The effective ratio {finite_ratio:.2%}")

    except Exception as e:
        bad[name] = f"Error: {repr(e)}"
        print(f"{name}: Fail to calculate -> {bad[name]}")

alpha001: The effective ratio 95.66%
alpha002: The effective ratio 100.00%
alpha003: The effective ratio 100.00%
alpha004: The effective ratio 96.70%
alpha006: The effective ratio 100.00%
alpha007: The effective ratio 94.39%
alpha008: The effective ratio 96.21%
alpha011: The effective ratio 97.12%
alpha012: The effective ratio 97.28%
alpha013: The effective ratio 97.04%
alpha014: The effective ratio 97.12%
alpha015: The effective ratio 99.84%
alpha016: The effective ratio 97.03%
alpha017: The effective ratio 95.48%
alpha018: The effective ratio 97.03%
alpha019: delete (The effective value ratio is too low：77.36%)
alpha020: The effective ratio 97.28%
alpha022: The effective ratio 95.81%
alpha025: The effective ratio 95.80%
alpha026: The effective ratio 99.84%
alpha028: The effective ratio 97.36%
alpha029: The effective ratio 96.54%
alpha030: The effective ratio 95.81%
alpha032: delete (The effective value ratio is too low：78.43%)
alpha033: The effective ratio 97.36%
alpha034: The effect

In [15]:
factors = pd.concat(usable, axis=1)
factors.index.name = 'date'
factors.columns.names = ['alpha', 'Name']

In [17]:
# cross-sectional z-score calculation 
def cs_zscore(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize each day's cross-section of values across stocks.
    We treat each row as one trading day and each column as one stock.
    For every date t, we compute:
        z_{i,t} = (x_{i,t} - mean_t) / std_t
    where mean_t and std_t are the cross-sectional mean and standard
    deviation across all stocks on day t.
    
    This makes the values comparable across time and focuses the model
    on relative differences between stocks on the same day, rather
    than on the absolute level of the variable.
    """
    # Cross-sectional mean and standard deviation across stocks (columns)
    mean = df.mean(axis=1)
    std = df.std(axis=1)

    # If for some day all stocks have exactly the same value, the
    # cross-sectional std would be 0.  To avoid division by zero, we
    # replace 0 with 1 so that the corresponding z-scores become 0.
    std = std.replace(0, 1)
    
    # Subtract the daily cross-sectional mean and divide by the
    # daily cross-sectional standard deviation.
    return df.sub(mean, axis=0).div(std, axis=0)


# We start from 'close' prices and compute log returns over
# a horizon of H trading days for each stock:
# r_{i, t to t+H} = log(P_{i, t+H}) − log(P_{i, t})

# Intuitively, this measures how much stock i goes up or down over the
# next H days, on a log scale, starting from day t.
H = 5 

symbols = sorted(factors.columns.get_level_values('Name').unique())
close_sub = close[symbols].sort_index()

log_price = np.log(close_sub)
log_ret_fwd = log_price.shift(-H) - log_price 
y_raw = log_ret_fwd

# As in the factor construction, we standardized the forward returns
# cross-sectionally. This means that for each day t, we converted the
# raw returns into z-scores across stocks. The model is therefore
# trained to predict which stocks will perform better or worse
# relative to other stocks on the same day, rather than predicting
# the absolute size of returns.
y = cs_zscore(y_raw)

In [19]:
selected_alphas = sorted(usable.keys())
print("Number of valid alpha factors:", len(selected_alphas), selected_alphas[:10], "...")

def cs_zscore_factor_panel(factors_dict):
    """
    Apply cross-sectional z-score normalization to each alpha factor panel
    and stack them into a single MultiIndex DataFrame.
    """
    z_dict = {}
    for name, fac in factors_dict.items():
        # Make sure rows are sorted by date for this factor
        fac = fac.sort_index()
        
        # For this factor, standardize each day's cross-section across stocks
        fac_z = cs_zscore(fac)          
        z_dict[name] = fac_z

    # first level = factor name and second level = stock name.
    out = pd.concat(z_dict, axis=1)
    out.index.name = 'date'
    out.columns.names = ['alpha', 'Name']
    return out

# Build the full standardized factor panel:
# rows = dates, columns = (alpha, stock), values = z-scored factor values.
factors_z = cs_zscore_factor_panel(usable)
factors_z.iloc[300:305].head()

Number of valid alpha factors: 36 ['alpha001', 'alpha002', 'alpha003', 'alpha004', 'alpha006', 'alpha007', 'alpha008', 'alpha011', 'alpha012', 'alpha013'] ...


alpha       alpha001                                                    \
Name               A       AAL       AAP      AAPL      ABBV       ABC   
date                                                                     
2014-04-21  1.010930  1.010930  1.010930  1.010930  1.010930 -1.079597   
2014-04-22  0.892475  0.892475  0.892475  0.892475  0.892475 -1.414709   
2014-04-23  1.205370  1.205370  1.205370  0.048364  1.205370 -1.525314   
2014-04-24  1.139743  1.139743  1.139743  1.139743  0.141537  1.139743   
2014-04-25  0.647243  0.647243  0.647243  1.467329 -0.202263  0.647243   

alpha                                               ...  alpha101            \
Name             ABT       ACN      ADBE       ADI  ...        XL      XLNX   
date                                                ...                       
2014-04-21 -0.217445  1.010930  1.010930 -1.632748  ... -0.620463 -0.263020   
2014-04-22 -0.904166 -0.397521  0.892475  0.892475  ... -0.158113 -0.099065   
2014-04-23 -1.112363 -0.744056  0.048364  0.048364  ...  2.066905  0.295582   
2014-04-24 -1.657469 -1.322250 -0.662987  1.139743  ...  0.208201 -1.307533   
2014-04-25 -0.202263 -1.585009 -0.967187  0.647243  ... -0.116334  1.059119   

alpha                                                                   \
Name             XOM      XRAY       XRX       XYL       YUM       ZBH   
date                                                                     
2014-04-21  1.740996 -0.377818 -0.735473  0.602375  1.095045 -0.297801   
2014-04-22 -0.961593 -1.351729  0.976544  0.335605  1.450526 -1.545718   
2014-04-23 -0.203261 -0.029474  1.229480  0.576228 -1.291116 -0.016322   
2014-04-24 -1.001937 -1.572670 -0.055063  0.084266 -0.481236 -1.100133   
2014-04-25  1.901715 -0.045189 -1.062140 -0.089338  2.100520  0.298683   

alpha                           
Name            ZION       ZTS  
date                            
2014-04-21 -0.285868  1.183843  
2014-04-22  1.489270  0.618461  
2014-04-23  0.645839 -1.484964  
2014-04-24 -1.663658  2.090296  
2014-04-25  0.274443  1.639684  

[5 rows x 18180 columns]

In [21]:
# Lookback window length (number of past trading days used as input)
L = 20    

# Forward return horizon in days
H = 5           
alpha_list = selected_alphas
symbols = sorted(factors_z.columns.get_level_values('Name').unique())

X_list = []
y_list = []
date_list = []
sym_list = []

for sym in symbols:
    # Extract the factor panel for a single stock:
    # rows = dates, columns = alpha factors
    fac_sym = (
        factors_z
        # select this stock across all alpha factors
        .xs(sym, axis=1, level='Name')   
        [alpha_list]
        .sort_index()
    )

    # Extract the corresponding H-day forward return labels for this stock:
    # y_sym[t] is the z-scored forward H-day log return starting from date t.
    y_sym = y[sym].sort_index()

    fac_arr = fac_sym.to_numpy()  
    y_arr = y_sym.to_numpy()      
    dates_sym = fac_sym.index.to_numpy()

    T = fac_arr.shape[0]

    # For each date t, we build one training sample:
    # Input X_t: the factor history for this stock over the last L days, (i.e., rows [t-L+1, ..., t] of fac_arr).
    # Target y_t: the forward H-day z-scored log return starting at date t.
    # To make sure we have L days of history and H days into the future,
    for t in range(L - 1, T - H):
         # Factor sequence window of length L ending at date t
        window_X = fac_arr[t - L + 1:t + 1, :]  
        # Corresponding forward H-day return label
        target = y_arr[t]                        

        if not np.isfinite(window_X).all():
            continue
        if not np.isfinite(target):
            continue

        X_list.append(window_X)
        y_list.append(target)
        date_list.append(dates_sym[t])   
        sym_list.append(sym)

X = np.stack(X_list).astype(np.float32)      
y_vec = np.array(y_list, dtype=np.float32)  

dates_arr = np.array(date_list)
sym_arr = np.array(sym_list)

X.shape, y_vec.shape

((503609, 20, 36), (503609,))

In [23]:
# We split the data into train, validation, and test data set.

train_end = pd.Timestamp('2016-1-31')
val_end   = pd.Timestamp('2017-1-31')

train_mask = dates_arr <= train_end
val_mask   = (dates_arr > train_end) & (dates_arr <= val_end)
test_mask  = dates_arr > val_end

print("Train:", train_mask.sum(), "Val:", val_mask.sum(), "Test:", test_mask.sum())

X_train, y_train = X[train_mask], y_vec[train_mask]
X_val,   y_val   = X[val_mask],   y_vec[val_mask]
X_test,  y_test  = X[test_mask],  y_vec[test_mask]

Train: 254118 Val: 123966 Test: 125525


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

# Convert X from shape (N, L, K) to (N, L*K) so that it can be used by LinearRegression.
def flatten_seq(X):
    return X.reshape(X.shape[0], -1)

np.random.seed(6)
X_train_flat = flatten_seq(X_train)
X_val_flat   = flatten_seq(X_val)
X_test_flat  = flatten_seq(X_test)

ols = LinearRegression()
ols.fit(X_train_flat, y_train)

y_pred_train = ols.predict(X_train_flat)
y_pred_val   = ols.predict(X_val_flat)
y_pred_test  = ols.predict(X_test_flat)


# Report mean squared error (MSE) on train, validation, and test sets.
# This evaluates how close the predicted z-scored returns are to the true
# z-scored returns in a regression sense.
print("OLS MSE train:", mean_squared_error(y_train, y_pred_train))
print("OLS MSE val  :", mean_squared_error(y_val,   y_pred_val))
print("OLS MSE test :", mean_squared_error(y_test,  y_pred_test))


# Extract the dates corresponding to each test sample, so we can compute
# cross-sectional performance (RankIC) date by date.
dates_test = dates_arr[test_mask]

unique_dates = np.unique(dates_test)
ic_list = []

for d in unique_dates:
     # Select all test samples for this trading day (all stocks).
    mask_d = (dates_test == d)
    yt_d = y_test[mask_d]
    yp_d = y_pred_test[mask_d]

    # Remove any non-finite values to avoid issues in Spearman correlation
    valid = np.isfinite(yt_d) & np.isfinite(yp_d)
    yt_d = yt_d[valid]
    yp_d = yp_d[valid]

    # RankIC is defined as the cross-sectional Spearman rank correlation
    # between predicted and realized returns on a given day.
    # Intuitively, it measures how well the model ranks stocks from
    # "good" to "bad" relative performers on that day.
    ic = spearmanr(yt_d, yp_d).correlation
    if np.isfinite(ic):
        ic_list.append(ic)

if len(ic_list) == 0:
    print("We can not calculate the valid RankIC value for OLS")
else:
    ic_arr = np.array(ic_list)
    mean_ic = ic_arr.mean()
    std_ic  = ic_arr.std()
    print("The mean of 5-day RankIC on test set for OLS:", mean_ic)
    print("The sd of 5-day RankIC on test set for OLS:", std_ic)

OLS MSE train: 0.98660684
OLS MSE val  : 0.99592125
OLS MSE test : 1.0015659
The mean of 5-day RankIC on test set for OLS: 0.0024431491421802983
The sd of 5-day RankIC on test set for OLS: 0.07131201266705325


In [27]:
from sklearn.linear_model import Lasso 
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

# In this part, we use LASSO regression: linear model with L1 penalty.
# Compared to plain OLS, the L1 regularization encourages many
# coefficients to be exactly zero, effectively performing feature
# selection among the flattened factor-history inputs.
lasso = Lasso(alpha=0.01, max_iter=10000, random_state=6)
lasso.fit(X_train_flat, y_train)

y_pred_train_lasso = lasso.predict(X_train_flat)
y_pred_val_lasso   = lasso.predict(X_val_flat)
y_pred_test_lasso  = lasso.predict(X_test_flat)

print("LASSO MSE train:", mean_squared_error(y_train, y_pred_train_lasso))
print("LASSO MSE val  :", mean_squared_error(y_val,   y_pred_val_lasso))
print("LASSO MSE test :", mean_squared_error(y_test,  y_pred_test_lasso))


# RankIC evaluation (same as for OLS) 
# For each test date, we again compute, the Spearman rank correlation
# between predicted and realized returns across all stocks. This tells
# us how well LASSO ranks stocks from better to worse performers on
# each day, in a purely cross-sectional sense.
dates_test = dates_arr[test_mask]
unique_dates = np.unique(dates_test)
ic_list_lasso = []

for d in unique_dates:
    mask_d = (dates_test == d)
    yt_d = y_test[mask_d]
    yp_d = y_pred_test_lasso[mask_d]

    valid = np.isfinite(yt_d) & np.isfinite(yp_d)
    yt_d = yt_d[valid]
    yp_d = yp_d[valid]
    if len(yt_d) == 0:
        continue
    
    ic = spearmanr(yt_d, yp_d).correlation
    if np.isfinite(ic):
        ic_list_lasso.append(ic)

if len(ic_list_lasso) == 0:
    print("We can not calculate the valid RankIC value for LASSO")
else:
    ic_arr_lasso = np.array(ic_list_lasso)
    mean_ic_lasso = ic_arr_lasso.mean()
    std_ic_lasso  = ic_arr_lasso.std()
    print("The mean of 5-day RankIC on test set for LASSO:", mean_ic_lasso)
    print("The sd of 5-day RankIC on test set for LASSO:", std_ic_lasso)

LASSO MSE train: 0.993023
LASSO MSE val  : 0.9900943
LASSO MSE test : 0.99561924
The mean of 5-day RankIC on test set for LASSO: -0.00032719428906715714
The sd of 5-day RankIC on test set for LASSO: 0.09789243013900517


In [29]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FactorSequenceDataset(Dataset):
    """
    Each sample consists of:
        X ∈ R^{L × K} : a sequence of L days of factor values for one stock
                        (already cross-sectionally z-scored each day),
        y ∈ R         : the corresponding forward H-day return target for
                        the last day in the sequence (also z-scored
                        cross-sectionally across stocks).
    """
    def __init__(self, X, y):
        # X has shape (N, L, K) where:
        # N = number of (date, stock) samples
        # L = lookback length
        # K = number of alpha factors
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float().unsqueeze(-1)  # (N, 1)

    def __len__(self):
        # Total number of samples in the dataset
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 128

train_dataset = FactorSequenceDataset(X_train, y_train)
val_dataset   = FactorSequenceDataset(X_val,   y_val)
test_dataset  = FactorSequenceDataset(X_test,  y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

X_train.shape, y_train.shape

((254118, 20, 36), (254118,))

In [30]:
import torch.nn as nn

class FactorTransformer(nn.Module):
# Each input sample is a sequence of L days of K standardized factor values
# for a single stock, with shape (L, K).
    def __init__(
        self,
        num_features,   
        seq_len,        
        d_model=4,
        nhead=2,
        num_layers=2,
        dim_feedforward=16,
        dropout=0.25
    ):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model
        self.input_proj = nn.Linear(num_features, d_model)

        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.pos_embedding = nn.Parameter(torch.zeros(1, seq_len + 1, d_model))

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
            activation='gelu'
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.final_ln = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, 1)
        )

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.trunc_normal_(self.input_proj.weight, std=0.02)
        nn.init.zeros_(self.input_proj.bias)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embedding, std=0.02)
        for m in self.mlp:
            if isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.02)
                nn.init.zeros_(m.bias)

    def forward(self, x):

        bsz, L, K = x.shape
        assert L == self.seq_len, f"input sequence length {L} are not consistent with {self.seq_len}"

        h = self.input_proj(x)                       

        cls_tokens = self.cls_token.expand(bsz, -1, -1)  
        h = torch.cat([cls_tokens, h], dim=1)           

        h = h + self.pos_embedding[:, :L+1, :]

        h_enc = self.encoder(h)                        
        cls_out = h_enc[:, 0, :]                       
        cls_out = self.final_ln(cls_out)
        out = self.mlp(cls_out)                        
        return out


In [31]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

K = X_train.shape[2]
L = X_train.shape[1]

# Instantiate the Transformer model with relatively small capacity
model = FactorTransformer(
    num_features=K,
    seq_len=L,
    d_model=4,
    nhead=2,
    num_layers=2,       
    dim_feedforward=8,
    dropout=0.25        
).to(device)


# Mean squared error loss between predicted and true z-scored returns
criterion = nn.MSELoss()
# define the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)

def run_epoch(loader, model, optimizer=None, max_grad_norm=1.0):
    if optimizer is None:
        model.eval()
    else:
        model.train()

    total_loss = 0.0
    total_n = 0

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        if optimizer is not None:
            optimizer.zero_grad()

        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)

        if optimizer is not None:
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()

        total_loss += loss.item() * X_batch.size(0)
        total_n += X_batch.size(0)

    return total_loss / total_n

num_epochs = 10
best_val_loss = float('inf')
best_state = None

for epoch in range(1, num_epochs + 1):
    train_loss = run_epoch(train_loader, model, optimizer)
    val_loss   = run_epoch(val_loader,   model, optimizer=None)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

    print(f"Epoch {epoch:02d} | train MSE = {train_loss:.6f}")

# Restore the best-performing model
if best_state is not None:
    model.load_state_dict(best_state)
    model.to(device)


device: cuda
Epoch 01 | train MSE = 0.993467
Epoch 02 | train MSE = 0.993229
Epoch 03 | train MSE = 0.992899
Epoch 04 | train MSE = 0.992512
Epoch 05 | train MSE = 0.992393
Epoch 06 | train MSE = 0.992245
Epoch 07 | train MSE = 0.992255
Epoch 08 | train MSE = 0.992169
Epoch 09 | train MSE = 0.992073
Epoch 10 | train MSE = 0.991960


In [35]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr
import numpy as np

model.eval()
y_true_list = []
y_pred_list = []


# Collect predictions and true targets on the test set
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        y_hat = model(X_batch)

        y_true_list.append(y_batch.cpu().numpy().ravel())
        y_pred_list.append(y_hat.cpu().numpy().ravel())

y_true_all = np.concatenate(y_true_list)
y_pred_all = np.concatenate(y_pred_list)

# This is Overall regression error (MSE) on the test set
# This treats all (stock, date) observations as independent and measures
# the average squared difference between predicted and true z-scored forward returns.
mse_test = mean_squared_error(y_true_all, y_pred_all)
print("Transformer test MSE:", mse_test)

# We here compute RankIC for each test date, RankIC_t = Spearman rank correlation(pred_t, true_t)
# across all stocks, and then average these daily RankIC values.
dates_test = dates_arr[test_mask]

unique_dates = np.unique(dates_test)
ic_list = []

for d in unique_dates:
    mask_d = (dates_test == d)
    yt_d = y_true_all[mask_d]
    yp_d = y_pred_all[mask_d]

    # Remove any NaN / infinite values
    valid = np.isfinite(yt_d) & np.isfinite(yp_d)
    yt_d = yt_d[valid]
    yp_d = yp_d[valid]

    # Require at least a few stocks for the cross-sectional correlation
    if yt_d.size < 5:
        continue

    # If the model (or the true returns) are essentially constant on this day,
    # Spearman correlation is not meaningful; we skip such days.
    if np.allclose(yp_d, yp_d[0]) or np.allclose(yt_d, yt_d[0]):
        continue

    ic = spearmanr(yt_d, yp_d).correlation
    if np.isfinite(ic):
        ic_list.append(ic)

if len(ic_list) == 0:
    print("On all test days, the predicted or true values were too close to constants to calculate RankIC.")
else:
    mean_ic = np.mean(ic_list)
    std_ic  = np.std(ic_list)
    print("The mean of 5-day RankIC on test set for Transformer:", mean_ic)
    print("The sd of 5-day RankIC on test set for Transformer:", std_ic)

Transformer test MSE: 0.9956638
The mean of 5-day RankIC on test set for Transformer: -0.0012800206223629912
The sd of 5-day RankIC on test set for Transformer: 0.12254856610823486


In [37]:
# Map the panel y_raw to a sample-level vector
# For each sample (date, symbol) pair, extract the corresponding 5-day
# forward log return from the panel y_raw, and flatten into a 1D array.
y_raw_vec = np.array(
    [y_raw.loc[d, s] for d, s in zip(dates_arr, sym_arr)],
    dtype=np.float32
)

# Keep only the test-set samples
# True 5-day log returns for the test set
y_raw_test = y_raw_vec[test_mask]       
dates_test = dates_arr[test_mask]       

# OLS: the test-set predictions were computed earlier as y_pred_test
y_pred_test_ols = y_pred_test

# LASSO: the test-set predictions were computed earlier as y_pred_test_lasso.

# In our earlier analysis, LASSO tended to have a negative RankIC, meaning
# that higher predicted scores were associated with lower realized returns.
# For portfolio construction, we want "higher score = more attractive stock",
# so we invert the sign here before using the scores for ranking.
y_pred_test_lasso = -y_pred_test_lasso

# For Transformer model, the test-set predictions were collected into y_pred_all
# when we ran the model over test_loader. Similarly, the Transformer’s
# raw scores also showed a negative RankIC, so we invert the sign so
# that larger scores correspond to better expected returns in the
# long–short portfolio ranking.
y_pred_test_trans = -y_pred_all

print(f"Number of test samples: {y_raw_test.shape[0]}")
print(f"OLS prediction length      : {y_pred_test_ols.shape[0]}")
print(f"LASSO prediction length    : {y_pred_test_lasso.shape[0]}")
print(f"Transformer prediction length: {y_pred_test_trans.shape[0]}")


# Long–short portfolio backtest
def long_short_portfolio_stats(y_raw_test, y_pred_test, dates_test,
                               top_q=0.2, bottom_q=0.2, H=5):
    """
    Construct a long–short portfolio based on model scores and compute
    performance statistics.

    On each test date t:
        - Rank all stocks by the model score s_{i,t} (higher is better).
        - Go long the top top_q fraction (e.g., top 20%) of stocks.
        - Go short the bottom bottom_q fraction (e.g., bottom 20%) of stocks.
        - For each stock, use the realized 5-day forward log return r_{i,t→t+H}
          to measure performance over the H-day holding period.
        - The long–short return for date t is the average simple return of the
          long portfolio minus the average simple return of the short portfolio.
    """

    unique_dates = np.unique(dates_test)
    ls_ret_list = []
    date_list   = []

    for d in unique_dates:
        # All test samples (stocks) corresponding to this calendar date
        mask_d = (dates_test == d)

        # 5-day forward log returns (true)
        r_d = y_raw_test[mask_d]   
        s_d = y_pred_test[mask_d] 

        # Remove NaN / non-finite values
        valid = np.isfinite(r_d) & np.isfinite(s_d)
        r_d = r_d[valid]
        s_d = s_d[valid]

        # Skip days with too few stocks to form meaningful portfolios
        if r_d.size < 10:
            continue

        # Sort stocks by Alpha factor score in descending order
        order = np.argsort(s_d)[::-1]
        n = r_d.size
        n_long  = int(np.floor(n * top_q))
        n_short = int(np.floor(n * bottom_q))

        if n_long == 0 or n_short == 0:
            continue

        r_long  = r_d[order[:n_long]]
        r_short = r_d[order[-n_short:]]

         # Convert 5-day log returns to simple returns: R = exp(r) - 1
        ret_long  = np.exp(r_long)  - 1.0
        ret_short = np.exp(r_short) - 1.0

        long_port_ret  = ret_long.mean()
        short_port_ret = ret_short.mean()
        ls_ret = long_port_ret - short_port_ret   

        ls_ret_list.append(ls_ret)
        date_list.append(d)

    ls_ret = np.array(ls_ret_list)
    dates_out = np.array(date_list)

    if ls_ret.size == 0:
        print("There is no effective sequence of long and short returns.")
        return None

    # Cumulative return curve of the long–short portfolio
    cum_curve = (1.0 + ls_ret).cumprod()

    # Annualization: treat each ls_ret as the return for one H-day holding period
    periods_per_year = 252.0 / H

    mean_ret = ls_ret.mean()
    vol      = ls_ret.std(ddof=1)

    ann_return = (1.0 + mean_ret) ** periods_per_year - 1.0
    ann_vol    = vol * np.sqrt(periods_per_year)
    sharpe     = ann_return / ann_vol if ann_vol > 0 else np.nan

    # Maximum drawdown based on the cumulative curve
    peak = np.maximum.accumulate(cum_curve)
    drawdown = cum_curve / peak - 1.0
    max_dd = drawdown.min()

    stats = {
        "dates": dates_out,      
        "ls_ret": ls_ret,        
        "cum_curve": cum_curve,  
        "ann_return": ann_return,
        "ann_vol": ann_vol,
        "sharpe": sharpe,
        "max_drawdown": max_dd,
    }
    return stats

# Long–short backtest for OLS / LASSO / Transformer
stats_ols    = long_short_portfolio_stats(y_raw_test, y_pred_test_ols,    dates_test, H=H)
stats_lasso  = long_short_portfolio_stats(y_raw_test, y_pred_test_lasso,  dates_test, H=H)
stats_trans  = long_short_portfolio_stats(y_raw_test, y_pred_test_trans,  dates_test, H=H)

print(" OLS long-short portfolio performance (Top 20% vs Bottom 20%, 5-day holding)")
print("Annual Return:", stats_ols["ann_return"])
print("Annual Volatility:", stats_ols["ann_vol"])
print("Sharpe   :", stats_ols["sharpe"])
print("Max Drawdown:", stats_ols["max_drawdown"])

print("\n LASSO long-short portfolio performance (Top 20% vs Bottom 20%, 5-day holding)")
print("Annual Return:", stats_lasso["ann_return"])
print("Annual Volatility:", stats_lasso["ann_vol"])
print("Sharpe   :", stats_lasso["sharpe"])
print("Max Drawdown:", stats_lasso["max_drawdown"])

print("\n Transformer long-short portfolio performance (Top 20% vs Bottom 20%, 5-day holding)")
print("Annual Return:", stats_trans["ann_return"])
print("Annual Volatility:", stats_trans["ann_vol"])
print("Sharpe   :", stats_trans["sharpe"])
print("Max Drawdown:", stats_trans["max_drawdown"])

Number of test samples: 125525
OLS prediction length      : 125525
LASSO prediction length    : 125525
Transformer prediction length: 125525
 OLS long-short portfolio performance (Top 20% vs Bottom 20%, 5-day holding)
Annual Return: -0.007968311624321212
Annual Volatility: 0.04406468730783651
Sharpe   : -0.18083213818481172
Max Drawdown: -0.20632374

 LASSO long-short portfolio performance (Top 20% vs Bottom 20%, 5-day holding)
Annual Return: -0.003195437912427823
Annual Volatility: 0.053181902075247575
Sharpe   : -0.06008506254452065
Max Drawdown: -0.16522682

 Transformer long-short portfolio performance (Top 20% vs Bottom 20%, 5-day holding)
Annual Return: 0.021126409528892554
Annual Volatility: 0.06853906348678927
Sharpe   : 0.30823895825428976
Max Drawdown: -0.21534818
